In [ ]:
import lmdb
from bigearthnet_patch_interface.s2_interface import BigEarthNet_S2_Patch
from pathlib import Path
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import Tensor, cat, stack

from torch.utils.data import DataLoader, ConcatDataset

In [ ]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

# Get paths
load_dotenv('./.env')

BEN_LMDB_PATH = os.environ.get("BEN_LMDB_PATH")
BEN_PATH = os.environ.get("BEN_PATH")

TRAIN_CSV_FILE = os.environ.get("TRAIN_CSV")
TEST_CSV_FILE = os.environ.get("TEST_CSV")
VAL_CSV_FILE = os.environ.get("VAL_CSV")

assert Path(BEN_PATH).exists()
assert Path(BEN_LMDB_PATH).exists()
assert os.path.exists(TRAIN_CSV_FILE)

In [ ]:
env = lmdb.open(BEN_LMDB_PATH, readonly=True, readahead=False, lock=False)
txn = env.begin()
cur = txn.cursor()

In [ ]:
import dataset_class as ds_class
importlib.reload(ds_class)

val_ds = ds_class.BenDataset(VAL_CSV_FILE, BEN_LMDB_PATH)
test_ds = ds_class.BenDataset(TEST_CSV_FILE, BEN_LMDB_PATH)
train_ds = ds_class.BenDataset(TRAIN_CSV_FILE, BEN_LMDB_PATH)

ds = ConcatDataset([val_ds, test_ds, train_ds])
ds_loader = DataLoader(ds, batch_size=1, shuffle=False)

print("Dataset size:", len(ds_loader))

#### Plot class distribution

In [ ]:
class_dist = torch.zeros([19], dtype=torch.uint8)
print(class_dist)
cnt = 0

for _, y in ds_loader:
    cnt += 1
    
    if cnt % 1000 == 0:
        print(cnt)
        print(class_dist)
    
    class_dist = torch.add(class_dist, y)
    
print(class_dist)
# tensor([[   0, 7827,    2, 7840,    0, 4846,  447,  395, 1230,  363, 5378,    0,
#           811,   12, 1713, 1703,  289, 5128, 2229]])